In [82]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller
import pandas as pd
import cx_Oracle

In [83]:
try:
    wd = webdriver.Chrome()
except:
    chromedriver_autoinstaller.install()
    chrome_version = chromedriver_autoinstaller.get_chrome_version()
    wd = webdriver.Chrome()


The form of our url: 'https://investidor10.com.br/acoes/itsa4/' --> 'https://investidor10.com.br/acoes/ticket_stock/'

In [84]:
companies=['itsa4','wege3','ggbr4','itub4','rani3','b3sa3','suzb3','ciel3','rpad3']

In [85]:
def get_stocks(companies: list):
    info=[]
    for i in range(0, len(companies)):
        try:
            url = f'https://investidor10.com.br/acoes/{companies[i]}/'
            wd.get(url=url)
            wait = WebDriverWait(wd, 10)

            #Indicadores

            name = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'name-company')))
            ticket = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'name-ticker')))

            market_value = None
            firm_value = None
            enterprise_value = None
            number_shares = None
            assets = None
            current_assets = None
            free_float_element = None
            cash_and_equivalent = None
            listing_segment_title = None
            tag_along = None

            #Search on the page: Father - child
            try:
                market_value = wait.until(EC.presence_of_element_located((By.XPATH, '//span[@class="title" and contains(text(), "Valor de mercado")]/following-sibling::span[@class="value"]')))
                firm_value = wait.until(EC.presence_of_element_located((By.XPATH, '//span[@class="title" and contains(text(), "Valor de firma")]/following-sibling::span[@class="value"]')))
                enterprise_value = wait.until(EC.presence_of_element_located((By.XPATH, '//span[@class="title" and contains(text(), "Patrimônio Líquido")]/following-sibling::span[@class="value"]')))
                number_shares = wait.until(EC.presence_of_element_located((By.XPATH, '//span[@class="title" and contains(text(), "Nº total de papeis")]/following-sibling::span[@class="value"]')))
                time.sleep(2)
                assets = wait.until(EC.presence_of_element_located((By.XPATH, '//span[@class="title" and contains(text(), "Ativos")]/following-sibling::span[@class="value"]')))
                current_assets = wait.until(EC.presence_of_element_located((By.XPATH, '//span[@class="title" and contains(text(), "Ativo Circulante")]/following-sibling::span[@class="value"]')))
                cash_and_equivalent = wait.until(EC.visibility_of_element_located((By.XPATH, '//span[@class="title" and contains(text(), "Disponibilidade")]/following-sibling::span[@class="value"]')))
                tag_along = wait.until(EC.visibility_of_element_located((By.XPATH, '//span[@class="title" and contains(text(), "Tag Along")]/following-sibling::span[@class="value"]')))
                listing_segment_title = wait.until(EC.visibility_of_element_located((By.XPATH, '//span[@class="title" and contains(text(), "Segmento de Listagem")]/following-sibling::span[@class="value"]')))
                free_float_element = wait.until(EC.presence_of_element_located((By.XPATH, '//span[@class="title" and contains(text(), "Free Float")]/following-sibling::span[@class="value"]')))
                average_daily_liquidity = wait.until(EC.presence_of_element_located((By.XPATH, '//span[@class="title" and contains(text(), "Liquidez Média Diária")]/following-sibling::span[@class="value"]')))
                sector = wait.until(EC.presence_of_element_located((By.XPATH, '//span[@class="title" and contains(text(), "Setor")]/following-sibling::span[@class="value"]')))
            except:
                pass
                
            #if found, then text, otherwise empty
            listing_segment_title_text = listing_segment_title.text if listing_segment_title else None
            free_float_text = free_float_element.text if free_float_element else None
            tag_along_text = tag_along.text if tag_along else None
            cash_and_equivalent_text=cash_and_equivalent.text if cash_and_equivalent else None
            market_value_text= market_value.text if market_value else None
            firm_value_text = firm_value.text if firm_value else None
            enterprise_value_text = enterprise_value.text if enterprise_value else None
            number_shares_text= number_shares.text if number_shares else None
            assets_text= assets.text if assets else None
            current_assets_text = current_assets.text if current_assets else None
            average_daily_liquidity_text = average_daily_liquidity.text if average_daily_liquidity else None
            sector_text = sector.text if sector else None

                
            
            info.append({'name': name.text, 'ticket': ticket.text.split('\n')[0],
                         'market value':market_value_text, 'firm_value':firm_value_text, 
                         'enterprise_value':enterprise_value_text,'number_shares':number_shares_text,
                         'assets':assets_text,
                         'current_assets': current_assets_text,
                         'cash_equivalent': cash_and_equivalent_text, 'listing_segment':listing_segment_title_text,'free_float':free_float_text,
                         'tag_along':tag_along_text,'average_daily_liquidity':average_daily_liquidity_text,'sector':sector_text})
        except:
            raise Exception('element not found')
            
    return  pd.DataFrame(info).to_excel('stock_data.xlsx',index=False)

In [86]:
get_stocks(companies=companies)

Data process

In [102]:
stocks_information = pd.read_excel('stock_data.xlsx')

In [103]:
stocks_information

,name,ticket,market value,firm_value,enterprise_value,number_shares,assets,current_assets,cash_equivalent,listing_segment,free_float,tag_along,average_daily_liquidity,sector
0,ITAUSA INVESTIMENTOS ITAU S.A.,ITSA4,"R$ 106,83 Bilhões","R$ 109,65 Bilhões","R$ 79,74 Bilhões","10,33 Bilhões","R$ 102,82 Bilhões","R$ 12,68 Bilhões","R$ 8,21 Bilhões",Nível 1,"62,50%","80,00%","R$ 236,31 Milhões",Financeiro
1,WEG,WEGE3,"R$ 156,56 Bilhões","R$ 152,31 Bilhões","R$ 17,34 Bilhões","4,20 Bilhões","R$ 31,50 Bilhões","R$ 21,56 Bilhões","R$ 7,08 Bilhões",Novo Mercado,"35,30%","100,00%","R$ 251,06 Milhões",Bens Industriais
2,GERDAU,GGBR4,"R$ 38,08 Bilhões","R$ 43,63 Bilhões","R$ 49,06 Bilhões","1,76 Bilhão","R$ 74,89 Bilhões","R$ 29,20 Bilhões","R$ 5,34 Bilhões",Nível 1,"63,01%","100,00%","R$ 214,36 Milhões",Materiais Básicos
3,BANCO ITAU UNIBANCO,ITUB4,"R$ 311,22 Bilhões","R$ 311,22 Bilhões","R$ 190,18 Bilhões","9,80 Bilhões","R$ 2,54 Trilhões","R$ 158,48 Bilhões","R$ 32,00 Bilhões",Nível 1,"52,88%","80,00%","R$ 1,00 Bilhão",Financeiro
4,IRANI PAPEL E EMBALAGEM S.A,RANI3,"R$ 2,18 Bilhões","R$ 3,21 Bilhões","R$ 1,28 Bilhão","246,36 Milhões","R$ 3,47 Bilhões","R$ 1,13 Bilhão","R$ 600,98 Milhões",Tradicional,"41,44%","100,00%","R$ 12,04 Milhões",Materiais Básicos
5,"B3 S.A. - BRASIL, BOLSA, BALCÃO",B3SA3,"R$ 70,19 Bilhões","R$ 68,21 Bilhões","R$ 20,27 Bilhões","5,65 Bilhões","R$ 49,42 Bilhões","R$ 18,84 Bilhões","R$ 15,99 Bilhões",Nível 1,"98,53%","100,00%","R$ 523,13 Milhões",Financeiro
6,SUZANO S.A,SUZB3,"R$ 75,99 Bilhões","R$ 131,99 Bilhões","R$ 44,69 Bilhões","1,32 Bilhão","R$ 143,59 Bilhões","R$ 38,57 Bilhões","R$ 21,17 Bilhões",Novo Mercado,"54,89%","100,00%","R$ 215,59 Milhões",Materiais Básicos
7,CIELO,CIEL3,"R$ 14,67 Bilhões","R$ 21,15 Bilhões","R$ 12,28 Bilhões","2,72 Bilhões","R$ 107,78 Bilhões","R$ 96,75 Bilhões","R$ 1,25 Bilhão",Novo Mercado,"40,32%","100,00%","R$ 218,99 Milhões",Financeiro
8,ALFA HOLDINGS,RPAD3,"R$ 716,96 Milhões","R$ 656,45 Milhões","R$ 1,08 Bilhão","84,68 Milhões","R$ 1,09 Bilhão","R$ 62,48 Milhões","R$ 60,51 Milhões",Nível 1,"24,65%",-,"R$ 27,90 Mil",Financeiro


We're going to process the data to avoid the symbols: 'R$', ',',  'Bilhões' and '%'

In [104]:
def convert_currency(value):
    try:
        if 'Bilhões' in value:
            return float(value.replace('Bilhões', '').replace('R$', '').replace(',', '.')) * 1e9
        elif 'Bilhão' in value:
            return float(value.replace('Bilhão', '').replace('R$', '').replace(',', '.')) * 1e9
        elif 'Milhões' in value:
            return float(value.replace('Milhões', '').replace('R$', '').replace(',', '.')) * 1e6
        elif 'Mil' in value:
            return float(value.replace('Mil', '').replace('R$', '').replace(',', '.')) * 1e4
        elif 'Trilhões' in value:
            return float(value.replace('Trilhões', '').replace('R$', '').replace(',', '.')) * 1e12
        elif 'Trilhão' in value:
            return float(value.replace('Trilhão', '').replace('R$', '').replace(',', '.')) * 1e12
        else:
            return float(value.replace('R$', '').replace(',', '.'))
    except ValueError:
        return None

# List of columns to transform
currency_columns = ['market value', 'firm_value', 'enterprise_value', 'number_shares', 'assets', 'current_assets', 'cash_equivalent','average_daily_liquidity']

# Apply transformations to each column
for col in currency_columns:
    stocks_information[col] = stocks_information[col].apply(convert_currency)

In [105]:
def convert_percentage(value):
    try:
        # Replace comma with dot, remove percentage sign, then convert to float
        return float(value.replace(',', '').rstrip('%')) / 100.0
    except ValueError:
        print(f"Error converting value: {value}")
        return None  # or any default value you prefer

# List of percentage columns to transform
percentage_columns = ['free_float', 'tag_along']

# Apply transformations to percentage columns
for col in percentage_columns:
    stocks_information[col] = stocks_information[col].apply(convert_percentage)

Error converting value: -


In [106]:
def convert_number_shares(value):
    try:
        if isinstance(value, float):
            return value  # No need to convert if already a float
        elif 'Bilhões' in value:
            return float(value.replace('Bilhões', '').replace('R$', '').replace(',', '.')) * 1e9
        elif 'Milhões' in value:
            return float(value.replace('Milhões', '').replace('R$', '').replace(',', '.')) * 1e6
        elif 'Bilhão' in value:
            return float(value.replace('Bilhão', '').replace('R$', '').replace(',', '.')) * 1e9
        else:
            return float(value.replace('R$', '').replace(',', '.'))
    except ValueError:
        print(f"Error converting value: {value}")
        return None  # or any default value you prefer

# Apply the function to the 'number_shares' column
stocks_information['number_shares'] = stocks_information['number_shares'].apply(convert_number_shares)

In [107]:
pd.set_option('display.float_format', lambda x: '%.3f' % x) # view the entire number format

In [108]:
stocks_information

,name,ticket,market value,firm_value,enterprise_value,number_shares,assets,current_assets,cash_equivalent,listing_segment,free_float,tag_along,average_daily_liquidity,sector
0,ITAUSA INVESTIMENTOS ITAU S.A.,ITSA4,106830000000.000,109650000000.000,79740000000.000,10330000000.000,102820000000.000,12680000000.000,8210000000.000,Nível 1,62.500,80.000,236310000.000,Financeiro
1,WEG,WEGE3,156560000000.000,152310000000.000,17340000000.000,4200000000.000,31500000000.000,21560000000.000,7080000000.000,Novo Mercado,35.300,100.000,251060000.000,Bens Industriais
2,GERDAU,GGBR4,38080000000.000,43630000000.000,49060000000.000,1760000000.000,74890000000.000,29200000000.000,5340000000.000,Nível 1,63.010,100.000,214360000.000,Materiais Básicos
3,BANCO ITAU UNIBANCO,ITUB4,311220000000.000,311220000000.000,190180000000.000,9800000000.000,2540000000000.000,158480000000.000,32000000000.000,Nível 1,52.880,80.000,1000000000.000,Financeiro
4,IRANI PAPEL E EMBALAGEM S.A,RANI3,2180000000.000,3210000000.000,1280000000.000,246360000.000,3470000000.000,1130000000.000,600980000.000,Tradicional,41.440,100.000,12040000.000,Materiais Básicos
5,"B3 S.A. - BRASIL, BOLSA, BALCÃO",B3SA3,70190000000.000,68210000000.000,20270000000.000,5650000000.000,49420000000.000,18840000000.000,15990000000.000,Nível 1,98.530,100.000,523130000.000,Financeiro
6,SUZANO S.A,SUZB3,75990000000.000,131990000000.000,44690000000.000,1320000000.000,143590000000.000,38570000000.000,21170000000.000,Novo Mercado,54.890,100.000,215590000.000,Materiais Básicos
7,CIELO,CIEL3,14670000000.000,21150000000.000,12280000000.000,2720000000.000,107780000000.000,96750000000.000,1250000000.000,Novo Mercado,40.320,100.000,218990000.000,Financeiro
8,ALFA HOLDINGS,RPAD3,716960000.000,656450000.000,1080000000.000,84680000.000,1090000000.000,62480000.000,60510000.000,Nível 1,24.650,NaN,279000.000,Financeiro


Oracle connection

In [122]:
oracle_connection_str = 'hr/hr@localhost:1521/XEPDB1'
connection = cx_Oracle.connect(oracle_connection_str)
cursor= connection.cursor()

In [123]:
# Iterate through DataFrame rows and insert into Oracle table
for index, row in stocks_information.iterrows():
    try:
        print(f"Inserting row {index + 1}: {row}")
        cursor.execute("""
            INSERT INTO companies 
            VALUES (:name, :ticket, :market_value, :enterprise_value, :equity,
                    :number_shares, :assets, :current_assets, :listing_segment,
                    :free_float, :tag_along, :average_daily_liquidity, :sector)
        """, 
        name=row['name'] if not pd.isna(row['name']) else 'null',
        ticket=row['ticket'] if not pd.isna(row['ticket']) else 'null',
        market_value=row['market value'] if not pd.isna(row['market value']) else 0,
        enterprise_value=row['enterprise_value'] if not pd.isna(row['enterprise_value']) else 0,
        equity=row['firm_value'] if not pd.isna(row['firm_value']) else 0,
        number_shares=row['number_shares'] if not pd.isna(row['number_shares']) else 0,
        assets=row['assets'] if not pd.isna(row['assets']) else 0,
        current_assets=row['current_assets'] if not pd.isna(row['current_assets']) else 0,
        listing_segment=row['listing_segment'] if not pd.isna(row['listing_segment']) else 'null',
        free_float=row['free_float'] if not pd.isna(row['free_float']) else 0,
        tag_along=row['tag_along'] if not pd.isna(row['tag_along']) else 0,
        average_daily_liquidity=row['average_daily_liquidity'] if not pd.isna(row['average_daily_liquidity']) else 0,
        sector=row['sector'] if not pd.isna(row['sector']) else 'null'
        )
        connection.commit()
    except cx_Oracle.Error as e:
        print(f"Error inserting row {index + 1}: {e}")
# Close the cursor and connection
cursor.close()
connection.close()


Inserting row 1: name                       ITAUSA INVESTIMENTOS ITAU S.A.
ticket                                              ITSA4
market value                             106830000000.000
firm_value                               109650000000.000
enterprise_value                          79740000000.000
number_shares                             10330000000.000
assets                                   102820000000.000
current_assets                            12680000000.000
cash_equivalent                            8210000000.000
listing_segment                                   Nível 1
free_float                                         62.500
tag_along                                          80.000
average_daily_liquidity                     236310000.000
sector                                         Financeiro
Name: 0, dtype: object
Inserting row 2: name                                    WEG
ticket                                WEGE3
market value               156560000000.000